# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [4]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [5]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Shimoda,34.6667,138.9500,44.10,90,100,14.67,JP,1678008783
1,1,Faanui,-16.4833,-151.7500,81.55,79,99,11.68,PF,1678008784
2,2,Rikitea,-23.1203,-134.9692,79.65,75,100,14.85,PF,1678008784
3,3,Ushuaia,-54.8000,-68.3000,31.66,93,0,4.61,AR,1678008784
4,4,Pangnirtung,66.1451,-65.7125,31.73,81,100,12.95,CA,1678008784


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [6]:
%%capture --no-display

# Configure the map plot
city_humidity_map = city_data_df.hvplot.points("Lng", 
                                               "Lat", 
                                               geo = True, 
                                               tiles = "OSM", 
                                               frame_width = 700, 
                                               frame_height = 600, 
                                               size = "Humidity", 
                                               scale = 1, 
                                               color = "City", 
                                               hover_cols = ["City"])

# Display the map
city_humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [7]:
# Narrow down cities that fit *criteria and drop any results with null values 
#(*Criteria selected: Max Temp:27, Min Temp:21, Wind Speed:less than 4.5m/s, Cloudiness: Less than 15)
ideal_cities_weather = city_data_df[(city_data_df["Max Temp"] <27) & (city_data_df["Max Temp"] >21)
                                    & (city_data_df["Wind Speed"] >=4.5) & (city_data_df["Cloudiness"] >=30)]

# Drop any rows with null values
ideal_cities_weather.dropna()

# Display sample data
ideal_cities_weather

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
36,36,Upernavik,72.7868,-56.1549,22.78,84,100,20.04,GL,1678008789
52,52,Chumikan,54.7183,135.3150,24.76,97,97,5.44,RU,1678008791
63,63,Tasiilaq,65.6145,-37.6368,26.76,91,74,12.24,GL,1678008793
111,111,Severo-Kurilsk,50.6789,156.1250,22.50,72,87,9.42,RU,1678008799
136,136,Husavik,66.0449,-17.3389,26.94,90,100,8.08,IS,1678008803
179,179,Iqaluit,63.7506,-68.5145,24.53,100,100,4.61,CA,1678008808
188,188,Saint-Augustin,51.2260,-58.6502,21.45,90,99,18.10,CA,1678008809
330,330,Nome,64.5011,-165.4064,26.67,93,100,21.85,US,1678008643
358,358,Stepnyak,52.8339,70.7803,24.48,78,100,7.40,KZ,1678008832
361,361,Vokhtoga,58.8147,41.0461,26.46,90,100,7.92,RU,1678008819


### Step 3: Create a new DataFrame called `hotel_df`.

In [8]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_weather[["City","Country","Lat","Lng","Humidity"]].copy()

#Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
36,Upernavik,GL,72.7868,-56.1549,84,
52,Chumikan,RU,54.7183,135.3150,97,
63,Tasiilaq,GL,65.6145,-37.6368,91,
111,Severo-Kurilsk,RU,50.6789,156.1250,72,
136,Husavik,IS,66.0449,-17.3389,90,
179,Iqaluit,CA,63.7506,-68.5145,100,
188,Saint-Augustin,CA,51.2260,-58.6502,90,
330,Nome,US,64.5011,-165.4064,93,
358,Stepnyak,KZ,52.8339,70.7803,78,
361,Vokhtoga,RU,58.8147,41.0461,90,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [15]:
# Set parameters to search for a hotel
radius = 10000
categories = "accommodation.hotel"
params = {
    'categories': categories,
    'apiKey' : geoapify_key,
    'limit': 5
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    long = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{long},{lat},{radius}"
    params["bias"] = f"proximity:{long},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Upernavik - nearest hotel: No hotel found
Chumikan - nearest hotel: ТАТЬЯНА
Tasiilaq - nearest hotel: Angmagssalik
Severo-Kurilsk - nearest hotel: "Дом рыбака"
Husavik - nearest hotel: Fosshótel Húsavík
Iqaluit - nearest hotel: Frobisher Inn
Saint-Augustin - nearest hotel: No hotel found
Nome - nearest hotel: No hotel found
Stepnyak - nearest hotel: No hotel found
Vokhtoga - nearest hotel: No hotel found
Kyshtovka - nearest hotel: No hotel found
Mnogovershinnyy - nearest hotel: No hotel found
Listvyanskiy - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
36,Upernavik,GL,72.7868,-56.1549,84,No hotel found
52,Chumikan,RU,54.7183,135.3150,97,ТАТЬЯНА
63,Tasiilaq,GL,65.6145,-37.6368,91,Angmagssalik
111,Severo-Kurilsk,RU,50.6789,156.1250,72,"""Дом рыбака"""
136,Husavik,IS,66.0449,-17.3389,90,Fosshótel Húsavík
179,Iqaluit,CA,63.7506,-68.5145,100,Frobisher Inn
188,Saint-Augustin,CA,51.2260,-58.6502,90,No hotel found
330,Nome,US,64.5011,-165.4064,93,No hotel found
358,Stepnyak,KZ,52.8339,70.7803,78,No hotel found
361,Vokhtoga,RU,58.8147,41.0461,90,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [16]:
%%capture --no-display
## When set cloudiness: 15, then no hotels returned, so changed to 30 to see if return some hotels ##

# Configure the map plot
city_hotel_map = hotel_df.hvplot.points("Lng", 
                                        "Lat", 
                                         geo = True, 
                                         tiles = "OSM", 
                                         frame_width = 800, 
                                         frame_height = 500, 
                                         size = "Humidity", 
                                         scale = 1, 
                                         color = "City", 
                                         hover_cols = ["City", "Hotel Name", "Country"])
# Display the map
city_hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)